# Chess Position Analyzer -- Expects pgn to be on one line and no spaces after move number:
### This may require prepping the portable game notation (PGN) as it is versatile and allows for more variations of play 
### (like: FisherRandom or various time controls and tournament notations e.g.: Swiss or Round Robin).  Typically we can
### just filter out the extra information.

#### This notebook tests a neccesary core feature extraction from the portable game notation.  
#####  The position of each piece on the chess board is not readily availble from the list of moves.  However, the algebraic notation is fairly unambiguous and since, memory is rather cheap and widely availble: converting the moves to a set of positions for the sake of abstracting the moves into a wider context makes our search for novel associations to other games easier to query.  This association allows us to empirically test the relative strength of a position for one side or the other.

#### The first coding cell simply looks at the first game of the All Master Game (allMGs.pgn) in pgn format.
#####  The allMGs.pgn file was prepared from a repository of games that came with a chess engine called Hiracs, version released for 2016.  This game set goes back twenty years prior to 2016 or 1995.  The highly biased, currated set of top level games gradually increases in sample size over the years.  We need a game to test the feature extraction.  

In [7]:
fn = "allMGs.pgn"
f = open(fn)
line = f.read(553)
f.close()
print (line)


[Event "FIDE WGP Ankara 2012"]
[Site "Ankara TUR"]
[Date "2012.09.17"]
[Round "2"]
[White "Koneru, Humpy"]
[Black "Zhao, Xue"]
[Result "1/2-1/2"]
[BlackElo "2549"]
[ECO "A30"]
[EventDate "2012.09.16"]
[WhiteElo "2593"]

1.Nf3 Nf6 2.c4 e6 3.g3 b6 4.Bg2 Bb7 5.O-O c5 6.Nc3 Be7 7.d4 cxd4 8.Qxd4 d6
9.b3 Nbd7 10.Rd1 a6 11.e4 Qc8 12.Bb2 O-O 13.h3 Rd8 14.Qe3 Qc7 15.Rd2 Rac8
16.Nd4 Bf8 17.Re1 Qb8 18.Qe2 Re8 19.Nc2 Nc5 20.b4 Ncd7 21.f4 Red8 22.Red1 Qc7
23.Ne3 h6 24.a3 Nh7 25.h4 Be7 26.Bf3 Ndf8 27.Qf2 Qb8 28.Qe2 Qc7 29.Rd3 Qb8
30.R3d2 Qc7 31.Rd3 Qb8 32.R3d2 


#### Demo filter:
##### In the previous code block we simply repeated the content of the first game of the file.  Here we are actually filtering the moves from the commentary on the game.  Most games repeat the results found in the commentary at the end of the move list.  Since, we do not see the drawn result at the end of the move list we most likely need to read more bytes in order to reach that point in the move list.  As for now I simply want to code up the rules for the movement of the pieces as this is the key to extracting the placement of the pieces at each move of the game, aka: Forsyth-Edwards Notation or FEN.
##### The last line demostrates the range of representational board positions for characters that we'll parse from the algebraic notation.

In [8]:
moves = ""
comments = ""
lines = line.split("\n")
for l in lines:
    if l.startswith("["):
        comments = comments + l
    else:
        moves = moves +" "+ l
print (moves)
print (comments)
print ("01-8:a-h["+str(ord('0'))+str(ord('1'))+"-"+str(ord('8'))+":"+str(ord('a'))+"-"+str(ord('h'))+"]")

  1.Nf3 Nf6 2.c4 e6 3.g3 b6 4.Bg2 Bb7 5.O-O c5 6.Nc3 Be7 7.d4 cxd4 8.Qxd4 d6 9.b3 Nbd7 10.Rd1 a6 11.e4 Qc8 12.Bb2 O-O 13.h3 Rd8 14.Qe3 Qc7 15.Rd2 Rac8 16.Nd4 Bf8 17.Re1 Qb8 18.Qe2 Re8 19.Nc2 Nc5 20.b4 Ncd7 21.f4 Red8 22.Red1 Qc7 23.Ne3 h6 24.a3 Nh7 25.h4 Be7 26.Bf3 Ndf8 27.Qf2 Qb8 28.Qe2 Qc7 29.Rd3 Qb8 30.R3d2 Qc7 31.Rd3 Qb8 32.R3d2 
[Event "FIDE WGP Ankara 2012"][Site "Ankara TUR"][Date "2012.09.17"][Round "2"][White "Koneru, Humpy"][Black "Zhao, Xue"][Result "1/2-1/2"][BlackElo "2549"][ECO "A30"][EventDate "2012.09.16"][WhiteElo "2593"]
01-8:a-h[4849-56:97-104]


#### Demo feature extraction: convert move (algebraic notation) to position (FEN)
##### Our goal is to query similar positions and rank the next move or candidate moves from a given position, empirically.  To this end we need not follow FEN precisely, such things as: "the right to castle" and "whose turn" are superfluous to our drive.  We simply want to capture the state of the board with respect to the location of pieces.  So, we need to encode the meaning of the move and track it from an initial board configuration through to the last move or end of the game; accounting for captures, castling and special moves (such as promotions and en pesant).

In [9]:
initial_state = "RRNNBBKQPPPPPPPPrrnnbbkqpppppppp/a8h8b8g8c8f8e8d8a7b7c7d7e7f7g7h7a1h1b1g1c1f1e1d1a2b2c2d2e2f2g2h2"
states = ["",initial_state]
PAWN = "P"
ROOK = "R"
KNIGHT = "N"
BISHOP = "B"
QUEEN = "Q"
KING = "K"
CAP = "x"
def queenMoves(fromHere, toHere, state):
    a = fromHere
    b = toHere
    i = 1
    while (i+ord(a[1]))<=ord('8'):
        if a[0] == b[0] and chr(ord(a[1])+i) == b[1]:
            return True
        else:
            if a[0]+chr(ord(a[1])+i) in state:
                break
        i+=1
    i = 1
    while (ord(a[0])-i)>=ord('a'):
        if chr(ord(a[0])-i) == b[0] and a[1] == b[1]:
            return True
        else:
            if chr(ord(a[0])-i)+a[1] in state:
                break
        i+=1
    i = 1
    while (i+ord(a[0]))<=ord('h'):
        if chr(ord(a[0])+i) == b[0] and a[1] == b[1]:
            return True
        else:
            if chr(ord(a[0])+i)+a[1] in state:
                break
        i+=1
    i = 1
    while (ord(a[1])-i)>=ord('1'):
        if a[0] == b[0] and chr(ord(a[1])-i) == b[1]:
            return True
        else:
            if a[0]+chr(ord(a[1])-i) in state:
                break
        i+=1
    i = 1
    while (i+ord(a[0]))<=ord('h') and (i+ord(a[1]))<=ord('8'):
        if chr(ord(a[0])+i) == b[0] and chr(ord(a[1])+i) == b[1]:
            return True
        else:
            if chr(ord(a[0])+i)+chr(ord(a[1])+i) in state:
                break
        i+=1
    i = 1
    while (ord(a[0])-i)>=ord('a') and (i+ord(a[1]))<=ord('8'):
        if chr(ord(a[0])-i) == b[0] and chr(ord(a[1])+i) == b[1]:
            return True
        else:
            if chr(ord(a[0])-i)+chr(ord(a[1])+i) in state:
                break
        i+=1
    i = 0
    while (i+ord(a[0]))<=ord('h') and (ord(a[1])-i)>=ord('1'):
        if chr(ord(a[0])+i) == b[0] and chr(ord(a[1])-i) == b[1]:
            return True
        else:
            if chr(ord(a[0])+i)+chr(ord(a[1])-i) in state:
                break
        i+=1
    i = 1
    while (ord(a[0])-i)>=ord('a') and (ord(a[1])-i)>=ord('1'):
        if chr(ord(a[0])-i) == b[0] and chr(ord(a[1])-i) == b[1]:
            return True
        else:
            if chr(ord(a[0])-i)+chr(ord(a[1])-i) in state:
                break
        i+=1
    return False
def rookMoves(fromHere, toHere, state):
    a = fromHere
    b = toHere
    i = 1
    while (i+ord(a[1]))<=ord('8'):
        if a[0] == b[0] and chr(ord(a[1])+i) == b[1]:
            return True
        else:
            if a[0]+chr(ord(a[1])+i) in state:
                break
        i+=1
    i = 1
    while (ord(a[0])-i)>=ord('a'):
        if chr(ord(a[0])-i) == b[0] and a[1] == b[1]:
            return True
        else:
            if chr(ord(a[0])-i)+a[1] in state:
                break
        i+=1
    i = 1
    while (i+ord(a[0]))<=ord('h'):
        if chr(ord(a[0])+i) == b[0] and a[1] == b[1]:
            return True
        else:
            if chr(ord(a[0])+i)+a[1] in state:
                break
        i+=1
    i = 1
    while (ord(a[1])-i)>=ord('1'):
        if a[0] == b[0] and chr(ord(a[1])-i) == b[1]:
            return True
        else:
            if a[0]+chr(ord(a[1])-i) in state:
                break
        i+=1
    return False 
def bishopMoves(fromHere, toHere, state):
    a = fromHere
    b = toHere
    i = 1
    while (i+ord(a[0]))<=ord('h') and (i+ord(a[1]))<=ord('8'):
        if chr(ord(a[0])+i) == b[0] and chr(ord(a[1])+i) == b[1]:
            return True
        else:
            if chr(ord(a[0])+i)+chr(ord(a[1])+i) in state:
                break
        i+=1
    i = 1
    while (ord(a[0])-i)>=ord('a') and (i+ord(a[1]))<=ord('8'):
        if chr(ord(a[0])-i) == b[0] and chr(ord(a[1])+i) == b[1]:
            return True
        else:
            if chr(ord(a[0])-i)+chr(ord(a[1])+i) in state:
                break
        i+=1
    i = 1
    while (i+ord(a[0]))<=ord('h') and (ord(a[1])-i)>=ord('1'):
        if chr(ord(a[0])+i) == b[0] and chr(ord(a[1])-i) == b[1]:
            return True
        else:
            if chr(ord(a[0])+i)+chr(ord(a[1])-i) in state:
                break
        i+=1
    i = 1
    while (ord(a[0])-i)>=ord('a') and (ord(a[1])-i)>=ord('1'):
        if chr(ord(a[0])-i) == b[0] and chr(ord(a[1])-i) == b[1]:
            return True
        else:
            if chr(ord(a[0])-i)+chr(ord(a[1])-i) in state:
                break
        i+=1
    return False
def knightMoves(fromHere, toHere):
    a = fromHere
    b = toHere
    if (chr(2+ord(a[0]))==b[0] and chr(1+ord(a[1])) == b[1]):
        return True
    if (chr(-2+ord(a[0]))==b[0] and chr(-1+ord(a[1])) == b[1]):
        return True
    if (chr(-2+ord(a[0]))==b[0] and chr(1+ord(a[1])) == b[1]):
        return True
    if (chr(2+ord(a[0]))==b[0] and chr(-1+ord(a[1])) == b[1]):
        return True
    if (chr(-1+ord(a[0]))==b[0] and chr(2+ord(a[1])) == b[1]):
        return True
    if (chr(1+ord(a[0]))==b[0] and chr(2+ord(a[1])) == b[1]):
        return True
    if (chr(1+ord(a[0]))==b[0] and chr(-2+ord(a[1])) == b[1]):
        return True
    if (chr(-1+ord(a[0]))==b[0] and chr(-2+ord(a[1])) == b[1]):
        return True
    return False
def display(move, isWhite,states):
    PAWN = "P"
    ROOK = "R"
    KNIGHT = "N"
    BISHOP = "B"
    QUEEN = "Q"
    KING = "K"
    CAP = "x"
    state = states[-1]
    if 2 == move.count("-"):
        if isWhite:
            state = state.replace("a1","d1")
            state = state.replace("e1","c1")
            states.append(state)
            return 0
        else:
            state = state.replace("a8","d8")
            state = state.replace("e8","c8")
            states.append(state)
            return 0
    if 1 == move.count("-"):
        if isWhite:
            state = state.replace("h1","f1")
            state = state.replace("e1","g1")
            states.append(state)
            return 0
        else:
            state = state.replace("h8","f8")
            state = state.replace("e8","g8")
            states.append(state)
            return 0
    pwnDir = 1
    cap = ""
    psqr = ""
    nsqr = move[-2:]
    if 0>=len(move):
        frameinfo = getframeinfo(currentframe())
        return frameinfo.lineno
    piece = move[0]
    if piece != ROOK and piece != BISHOP and piece != KNIGHT and piece != KING and piece != QUEEN:
        piece = PAWN
    if isWhite:
        pwnDir = -1
        c = state.split("/")[0].count(piece.lower())
        if c == 0:
            frameinfo = getframeinfo(currentframe())
            return frameinfo.lineno
        if -1 == state.find("/"):
            frameinfo = getframeinfo(currentframe())
            return frameinfo.lineno
        i = state.split("/")[0].find(piece.lower())*2
        if i >= len(state.split("/")[1]):
            frameinfo = getframeinfo(currentframe())
            return frameinfo.lineno
        d = move[1]
        psqr = state.split("/")[1][i]+state.split("/")[1][i+1]
        if 2 == c and len(move) > 3 and d != CAP and -1 == psqr.find(d):
            psqr = state.split("/")[1][i+2]+state.split("/")[1][i+3]
        if piece == PAWN:
            psqr = move[0]+chr(pwnDir+ord(nsqr[1]))
            if -1 == state.find(psqr) and nsqr[1] == '4':
                psqr = nsqr[0]+'2'
        if piece == BISHOP and c>=2 and not bishopMoves(psqr,nsqr,state):
            psqr = state.split("/")[1][i+2]+state.split("/")[1][i+3]
        if piece == QUEEN and c>=2 and not queenMoves(psqr,nsqr,state):
            psqr = state.split("/")[1][i+2]+state.split("/")[1][i+3]
        if piece == ROOK and c>=2 and not rookMoves(psqr,nsqr,state):
            psqr = state.split("/")[1][i+2]+state.split("/")[1][i+3]
        if piece == KNIGHT and c>=2 and not knightMoves(psqr,nsqr):
            psqr = state.split("/")[1][i+2]+state.split("/")[1][i+3]
        if -1 != move.find(CAP):
            if -1 < state.split("/")[1].find(nsqr):
                cap = state[((int)(state.split("/")[1].find(nsqr)/2))]
                state = state.replace(cap,"",1)
                state = state.replace(nsqr,"",1)
            else:
                cap = state[((int)(state.split("/")[1].find(nsqr[0]+(chr(ord(nsqr[1])+pwnDir)))/2))]
                state = state.replace(cap,"",1)
                state = state.replace(nsqr[0]+(chr(ord(nsqr[1])+pwnDir)),"",1)
        state = state.replace(psqr,nsqr)
        states.append(state)
    else:
        c = state.split("/")[0].count(piece)
        if c == 0:
            frameinfo = getframeinfo(currentframe())
            return frameinfo.lineno
        i = state.split("/")[0].find(piece)*2
        if -1 == state.find("/"):
            frameinfo = getframeinfo(currentframe())
            return frameinfo.lineno
        if i >= len(state.split("/")[1]):
            frameinfo = getframeinfo(currentframe())
            return frameinfo.lineno
        d = move[1]
        psqr = state.split("/")[1][i]+state.split("/")[1][i+1]
        if 2 == c and len(move) > 3 and d != CAP and -1 == psqr.find(d):
            psqr = state.split("/")[1][i+2]+state.split("/")[1][i+3]
        if piece == PAWN:
            psqr = move[0]+(chr(pwnDir+ord(nsqr[1])))
            if -1 == state.find(psqr) and nsqr[1] == '5':
                psqr = nsqr[0]+'7'
        if piece == BISHOP and c>=2 and not bishopMoves(psqr,nsqr,state):
            psqr = state.split("/")[1][i+2]+state.split("/")[1][i+3]
        if piece == QUEEN and c>=2 and not queenMoves(psqr,nsqr,state):
            psqr = state.split("/")[1][i+2]+state.split("/")[1][i+3]
        if piece == ROOK and c>=2 and not rookMoves(psqr,nsqr,state):
            psqr = state.split("/")[1][i+2]+state.split("/")[1][i+3]
        if piece == KNIGHT and c>=2 and not knightMoves(psqr,nsqr):
            psqr = state.split("/")[1][i+2]+state.split("/")[1][i+3]
        if -1 != move.find(CAP):
            if -1 < state.split("/")[1].find(nsqr):
                cap = state[((int)(state.split("/")[1].find(nsqr)/2))]
                state = state.replace(cap,"",1)
                state = state.replace(nsqr,"",1)
            else:
                cap = state[((int)(state.split("/")[1].find(nsqr[0]+(chr(ord(nsqr[1])+pwnDir)))/2))]
                state = state.replace(cap,"",1)
                state = state.replace(nsqr[0]+(chr(ord(nsqr[1])+pwnDir)),"",1)
        state = state.replace(psqr,nsqr)
        states.append(state)
    print (state)
    return 0
print (len(moves))
for m in moves.split():
    print (m)
    if -1 < m.find("."):
        display(m.split(".")[1], True,states)
    else:
        display(m, False,states)


335
1.Nf3
RRNNBBKQPPPPPPPPrrnnbbkqpppppppp/a8h8b8g8c8f8e8d8a7b7c7d7e7f7g7h7a1h1b1f3c1f1e1d1a2b2c2d2e2f2g2h2
Nf6
RRNNBBKQPPPPPPPPrrnnbbkqpppppppp/a8h8b8f6c8f8e8d8a7b7c7d7e7f7g7h7a1h1b1f3c1f1e1d1a2b2c2d2e2f2g2h2
2.c4
RRNNBBKQPPPPPPPPrrnnbbkqpppppppp/a8h8b8f6c8f8e8d8a7b7c7d7e7f7g7h7a1h1b1f3c1f1e1d1a2b2c4d2e2f2g2h2
e6
RRNNBBKQPPPPPPPPrrnnbbkqpppppppp/a8h8b8f6c8f8e8d8a7b7c7d7e6f7g7h7a1h1b1f3c1f1e1d1a2b2c4d2e2f2g2h2
3.g3
RRNNBBKQPPPPPPPPrrnnbbkqpppppppp/a8h8b8f6c8f8e8d8a7b7c7d7e6f7g7h7a1h1b1f3c1f1e1d1a2b2c4d2e2f2g3h2
b6
RRNNBBKQPPPPPPPPrrnnbbkqpppppppp/a8h8b8f6c8f8e8d8a7b6c7d7e6f7g7h7a1h1b1f3c1f1e1d1a2b2c4d2e2f2g3h2
4.Bg2
RRNNBBKQPPPPPPPPrrnnbbkqpppppppp/a8h8b8f6c8f8e8d8a7b6c7d7e6f7g7h7a1h1b1f3c1g2e1d1a2b2c4d2e2f2g3h2
Bb7
RRNNBBKQPPPPPPPPrrnnbbkqpppppppp/a8h8b8f6b7f8e8d8a7b6c7d7e6f7g7h7a1h1b1f3c1g2e1d1a2b2c4d2e2f2g3h2
5.O-O
c5
RRNNBBKQPPPPPPPPrrnnbbkqpppppppp/a8h8b8f6b7f8e8d8a7b6c5d7e6f7g7h7a1f1b1f3c1g2g1d1a2b2c4d2e2f2g3h2
6.Nc3
RRNNBBKQPPPPPPPPrrnnbbkqpppppppp/a8h8b8f6b7f8e8d8a7b6c5d7e6f7g7

## We now want to make sure there are other options for getting to FEN:
#### chess may be installed:

In [12]:
%pip install chess

DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ckan-2.11.0a0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 26.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147779 sha256=10904555ea01d0b468fe607e71a1c2515a9bad7efe385d0f1336c8474dce836b
  Stored in directory: /Users/edgarjohnson/Library/Caches/pip/wheels/fb/5d/5c/59a62d8a695285e59ec9c1f66add6f8a9ac4152499a2be0113
Successfully built chess

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#### if the system is missing a local sqlite db then we can install it as well:

In [ ]:
%pip install sqlite3

#### Some machine learning extensions can be found in mlxtend and scikit-learn

In [ ]:
%pip install mlxtend scikit-learn

DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ckan-2.11.0a0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 43.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.1
    Uninstalling joblib-1.1.1:
      Successfully uninstalled joblib-1.1.1
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-pr

#### The chess provides an api for managing the state of the board:
##### Storing the results of reading a large file into a persistent storage such as sqlite allows for more expressivity with the query capabilities and keeps us from having to keep reading the file.
##### I've added hashlib for the option of securing some features of the db.

In [13]:
import sqlite3
import hashlib
import chess
import chess.pgn
from  sqlite3  import  Error
from hashlib import blake2b

##### We create a connection to a flat db file for storing converted positions:

In [ ]:
def create_connection (db_file) :
    conn =  None
    try :
        conn = sqlite3.connect(db_file)
        return  conn
    except  Error  as  e:
        print(e)
    return  conn


#### This is a generic helper function that will create a table according to the schema passed in on the connection passed in:

In [ ]:
def create_table (conn, create_table_sql) :
    try :
        c = conn.cursor()
        c.execute(create_table_sql)
    except  Error  as  e:
        print(e)

#### The following defines the table schema for inserting converted positions into the local db:

In [ ]:
def create_chess_table ( database ) : 
    sql_create_positions_table =  """ CREATE TABLE IF NOT EXISTS positions (
                                        id integer PRIMARY KEY,
                                        move text NOT NULL,
                                        pposition text NOT NULL,
                                        rposition text NOT NULL,
                                        phash text,
                                        mhash text,
                                        total integer, 
                                        total01 integer,
                                        total10 integer,
                                        total12 integer
                                    ); """ 

    # create a database connection
    conn = create_connection(database)

    # create table
    if  conn  is   not   None :
        # create positions table 
        create_table(conn, sql_create_positions_table)
    else :
        print( "Error! cannot create the database connection." )
    return conn


#### The following treats the table as a hashmap counter, with the keys being the converted position and unique one-way hashes of the position and move.  The pposition or the state of the board prior to the move, is hashed in phash; while the resultting position after the move is stored into the rposition. +

In [ ]:
def insert_position (conn, pposition, move, rposition, result) : 
    h = blake2b(digest_size=16)
    h.update(pposition.encode())
    phash = h.hexdigest()
    h.update(move.encode())
    mhash = h.hexdigest()
    usql =  ''' UPDATE positions SET total = ?, total01 = ? , total10 = ? , total12 = ?  WHERE id = ?''' 
    isql =  ''' INSERT INTO positions(pposition,phash,move,mhash,rposition,total,total01,total10,total12)
              VALUES(?,?,?,?,?,?,?,?,?) ''' 
    cur = conn.cursor()
    cur.execute( "SELECT MAX(total), * FROM positions where mhash = '"+mhash+"' and phash = '"+phash+"';" )
    rows = cur.fetchall()
    if (len(rows)==0 or rows[0][0] == None):
      if result=="0-1": cur.execute(isql, (pposition,phash,move,mhash,rposition,1,1,0,0))
      if result=="1-0": cur.execute(isql, (pposition,phash,move,mhash,rposition,1,0,1,0))
      if result=="1/2-1/2": cur.execute(isql, (pposition,phash,move,mhash,rposition,1,0,0,1))
      conn.commit()
    else:
      print (rows)
      if result=="0-1": cur.execute(usql,(rows[0][7]+1,rows[0][8]+1,rows[0][9],rows[0][10],rows[0][1]))
      if result=="1-0": cur.execute(usql,(rows[0][7]+1,rows[0][8],rows[0][9]+1,rows[0][10],rows[0][1]))
      if result=="1/2-1/2": cur.execute(usql,(rows[0][7]+1,rows[0][8],rows[0][9],rows[0][10]+1,rows[0][1]))
      conn.commit()
    return  cur.lastrowid


#### Finally, we can create an entry-point that is based on an actual set of moves or games in pgn format and write the transformed positions to a sqllite file:

In [ ]:
def read_games () :
    database =  r"twic1450-2GamesPositions.db"
    conn = create_chess_table( database )
    pgn = open("twic1458.pgn",encoding='latin-1')

    # create a database connection create_connection(database) 
    with  conn:
      while True:
        headers = chess.pgn.read_headers(pgn)
        if headers is None:
          break
        game = chess.pgn.read_game(pgn)
        if None == game:
          continue
        board = game.board()
        for move in game.mainline_moves():
          out=board.fen()
          fin=out.split(" ")
          pposition=fin[0]
          alg=board.san(move)
          board.push(move)
          out=board.fen()
          fin=out.split(" ")
          rposition=fin[0]
          insert_position(conn,pposition,alg,rposition,headers["Result"])

#### We simply exercise the entry-point via a top-level call to read_games:

In [ ]:
read_games()

### Now we can build out the ingestion layer for loading into a sqlite db and begin empirically checking various selection criteria or smaller data sets for interesting similarities or patterns with games of interest.  We run this code in the background on the command line against a large dataset of pgn.  After it completes its execution we have a db file which we can explore in another tool for query and db management.

### Once we have prepared our data we can begin exploring / mining the data for hidden structures or frequent patterns and discern various groupings of interest.

In [14]:
from sklearn.tree import DecisionTreeClassifier
from mlxtend.frequent_patterns import apriori, association_rules
import pandas as pd
import sqlite3

#### The following graphic is from the arules visualizer available in R.  Association Rules in python requires a specific data format that requires an additional transformation before we can get to the rules similar to what we get out of the box with R.

![arules visualizer from R](RaRulesViz.png)

#### This graphic shows at a glance the strength of connections from a frequentist point of view.  This emperical way of teasing out interesting patterns and similarities in the data readily provides a set of moves commonly found together and even rarely seen combinations of moves.  I also was able to tease out from the SQLiteStudio a few positions of interest using the following sql:

###### select id, move, pposition, rposition, total01, total, CAST(total01 as REAL) / CAST(total as REAL) as m from positions where m > .8 and total > 10

#### This was found using from a set of games from a specific week in the collection of games. 

![Selection criteria](sqlitestudio.png)

In [18]:
#df = pd.DataFrame()

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('twic1450-2GamesPositions.db')

# Write the DataFrame to a SQLite table
#df.to_sql('positions', conn, if_exists='append', index=True)

# Verify by reading the table back into a DataFrame
df_from_db = pd.read_sql('SELECT * FROM positions', conn)

print(df_from_db)
X, y = df_from_db['rposition'], df_from_db['total10']/df_from_db['total']

# Create and train the decision tree classifier
clf = DecisionTreeClassifier()
clf = clf.fit(X, y)

# Plot the decision tree
plt.figure(figsize=(12, 8))
tree.plot_tree(clf, filled=True, feature_names=iris.feature_names, class_names=iris.target_names)
plt.show()

              id   move                                          pposition  \
0              1     e4        rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR   
1              2     e5      rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR   
2              3    Nf3    rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR   
3              4    Nc6  rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R   
4              5    Bb5  r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNB...   
...          ...    ...                                                ...   
2053460  2053461    Rg1         1r4k1/7p/p2pR3/2p2rp1/2P2P1b/1P5P/P6K/2BR4   
2053461  2053462    Kf7        1r4k1/7p/p2pR3/2p2rp1/2P2P1b/1P5P/P6K/2B3R1   
2053462  2053463    Rh6        1r6/5k1p/p2pR3/2p2rp1/2P2P1b/1P5P/P6K/2B3R1   
2053463  2053464    Re8        1r6/5k1p/p2p3R/2p2rp1/2P2P1b/1P5P/P6K/2B3R1   
2053464  2053465  Rxh7+        4r3/5k1p/p2p3R/2p2rp1/2P2P1b/1P5P/P6K/2B3R1   

                                                 rposition  \
0

ValueError: could not convert string to float: 'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR'